In [57]:
import os
import json
import openai
from datasets import load_dataset
from collections import OrderedDict

openai.api_key= os.environ["OPENAI_API_KEY"]

In [58]:
dataset = load_dataset("banking77")

Found cached dataset banking77 (/home/willy/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [59]:
test_data = dataset["test"]

In [60]:
test_data[57]

{'text': 'How do I view the card I received in the app?', 'label': 13}

In [61]:
for i in range(10, 15):
    print(i)

10
11
12
13
14


In [62]:
intent_labels = """0 	activate_my_card
1 	age_limit
2 	apple_pay_or_google_pay
3 	atm_support
4 	automatic_top_up
5 	balance_not_updated_after_bank_transfer
6 	balance_not_updated_after_cheque_or_cash_deposit
7 	beneficiary_not_allowed
8 	cancel_transfer
9 	card_about_to_expire
10 	card_acceptance
11 	card_arrival
12 	card_delivery_estimate
13 	card_linking
14 	card_not_working
15 	card_payment_fee_charged
16 	card_payment_not_recognised
17 	card_payment_wrong_exchange_rate
18 	card_swallowed
19 	cash_withdrawal_charge
20 	cash_withdrawal_not_recognised
21 	change_pin
22 	compromised_card
23 	contactless_not_working
24 	country_support
25 	declined_card_payment
26 	declined_cash_withdrawal
27 	declined_transfer
28 	direct_debit_payment_not_recognised
29 	disposable_card_limits
30 	edit_personal_details
31 	exchange_charge
32 	exchange_rate
33 	exchange_via_app
34 	extra_charge_on_statement
35 	failed_transfer
36 	fiat_currency_support
37 	get_disposable_virtual_card
38 	get_physical_card
39 	getting_spare_card
40 	getting_virtual_card
41 	lost_or_stolen_card
42 	lost_or_stolen_phone
43 	order_physical_card
44 	passcode_forgotten
45 	pending_card_payment
46 	pending_cash_withdrawal
47 	pending_top_up
48 	pending_transfer
49 	pin_blocked
50 	receiving_money
51 	Refund_not_showing_up
52 	request_refund
53 	reverted_card_payment?
54 	supported_cards_and_currencies
55 	terminate_account
56 	top_up_by_bank_transfer_charge
57 	top_up_by_card_charge
58 	top_up_by_cash_or_cheque
59 	top_up_failed
60 	top_up_limits
61 	top_up_reverted
62 	topping_up_by_card
63 	transaction_charged_twice
64 	transfer_fee_charged
65 	transfer_into_account
66 	transfer_not_received_by_recipient
67 	transfer_timing
68 	unable_to_verify_identity
69 	verify_my_identity
70 	verify_source_of_funds
71 	verify_top_up
72 	virtual_card_not_working
73 	visa_or_mastercard
74 	why_verify_identity
75 	wrong_amount_of_cash_received
76 	wrong_exchange_rate_for_cash_withdrawal"""
intent_labels = intent_labels.replace("\t", "")
intent_labels = intent_labels.split("\n")

label2intent = {}
intent2label = {}
for intent in intent_labels:
    temp_split = intent.split()
    label = temp_split[0]
    intent = temp_split[1]
    label2intent[label] = intent
    intent2label[intent] = label

In [12]:
INTENTS = ""
for label in label2intent:
    INTENTS += label + " " + label2intent[label] + "\n"

In [13]:
PROMPT_TEMPLATE_1 = f"""Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE, just generate the intent and nothing more:\n
INTENTS:
{INTENTS}
SENTENCE:"""

PROMPT_TEMPLATE_2 = f"""Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE and just generate the 2 most likely intent and nothing more:\n
INTENTS:
{INTENTS}
SENTENCE:"""

PROMPT_TEMPLATE_3 = f"""Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE and just generate the 3 most likely intent and nothing more:\n
INTENTS:
{INTENTS}
SENTENCE:"""

ALL_PROMPTS = {"top1":PROMPT_TEMPLATE_1,
               "top2":PROMPT_TEMPLATE_2,
               "top3":PROMPT_TEMPLATE_3}

In [14]:
preds = {}
for i in range(2):
    cur_preds = {}

    text_input = test_data[i]
    prompt = ALL_PROMPTS["top3"] + '"' + text_input["text"] + '"'

    cur_preds["text"] = text_input["text"]
    cur_preds["gold_label"] = text_input["label"]
    cur_preds["gold_intent"] = label2intent[str(text_input["label"])]
    cur_preds["prompt"] = prompt
    

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "user", "content": prompt}]
    )
    cur_preds["pred"] = completion["choices"][0]["message"]["content"]
    preds[i] = cur_preds

In [27]:
json_filename = "banking77_top3.json"

with open("results/" + json_filename, "w") as f:
    json.dump(preds, f, indent=4)

In [25]:
with open("results/" + json_filenam, "r") as f:
    preds = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
results = preds.copy()

In [37]:
for key, value in results.items():
    temp_result = {}
    outputs = value["pred"].split("\n")
    for output in outputs:
        if output[0] in ["1", "2", "3"]:
            temp_result["pred" + output[0]] = output.split(" ")[1]
        else:
            pass
    results[key]["intent_predicted"] = temp_result

processed_filename = json_filename[:-5] + "processed.json"
with open("results/" + processed_filename, "w") as f:
    json.dump(results, f, indent=4)

In [34]:
json_filename[:-5]

'banking77_top3'

In [16]:
results[0]

{'text': 'How do I locate my card?',
 'gold_label': 11,
 'gold_intent': 'card_arrival',
 'prompt': 'Given the following list of INTENTS, do intent classification on the sentence provided as SENTENCE and just generate the 3 most likely intent and nothing more:\n\nINTENTS:\n0 activate_my_card\n1 age_limit\n2 apple_pay_or_google_pay\n3 atm_support\n4 automatic_top_up\n5 balance_not_updated_after_bank_transfer\n6 balance_not_updated_after_cheque_or_cash_deposit\n7 beneficiary_not_allowed\n8 cancel_transfer\n9 card_about_to_expire\n10 card_acceptance\n11 card_arrival\n12 card_delivery_estimate\n13 card_linking\n14 card_not_working\n15 card_payment_fee_charged\n16 card_payment_not_recognised\n17 card_payment_wrong_exchange_rate\n18 card_swallowed\n19 cash_withdrawal_charge\n20 cash_withdrawal_not_recognised\n21 change_pin\n22 compromised_card\n23 contactless_not_working\n24 country_support\n25 declined_card_payment\n26 declined_cash_withdrawal\n27 declined_transfer\n28 direct_debit_payment_n

In [57]:
correct_count = {"pred1":0, "pred2":0, "pred3":0, "others":0}
for key, result in results.items():
    intents = result["intent_predicted"]
    gold = result["gold_intent"]
    found_gold = False
    for top, pred in intents.items():
        if pred not in intent2label:
            print(f"""This intent {pred} is not in the original intent list, it was in sample {key} from the test set and predicted as top {top[-1]}""")
        else:
            if pred == gold:
                correct_count[top] += 1
                found_gold = True
    if not found_gold:
        correct_count["others"] += 1
        print(key)
        
                
            

0
This intent getting_physical_card is not in the original intent list, it was in sample 2 from the test set and predicted as top 3
This intent getting_physical_card is not in the original intent list, it was in sample 3 from the test set and predicted as top 3


In [55]:
correct_count

{'pred1': 3, 'pred2': 1, 'pred3': 0, 'others': 1}

================================  
Observation:  

Interesting - Because in the intents there are some like "get_physical_cart" and "get-ting-_virtual_card", it creates confusion in the LLM. Inconsistency in labelling leads to inconsistency in prediction

In [49]:
intent2label

{'activate_my_card': '0',
 'age_limit': '1',
 'apple_pay_or_google_pay': '2',
 'atm_support': '3',
 'automatic_top_up': '4',
 'balance_not_updated_after_bank_transfer': '5',
 'balance_not_updated_after_cheque_or_cash_deposit': '6',
 'beneficiary_not_allowed': '7',
 'cancel_transfer': '8',
 'card_about_to_expire': '9',
 'card_acceptance': '10',
 'card_arrival': '11',
 'card_delivery_estimate': '12',
 'card_linking': '13',
 'card_not_working': '14',
 'card_payment_fee_charged': '15',
 'card_payment_not_recognised': '16',
 'card_payment_wrong_exchange_rate': '17',
 'card_swallowed': '18',
 'cash_withdrawal_charge': '19',
 'cash_withdrawal_not_recognised': '20',
 'change_pin': '21',
 'compromised_card': '22',
 'contactless_not_working': '23',
 'country_support': '24',
 'declined_card_payment': '25',
 'declined_cash_withdrawal': '26',
 'declined_transfer': '27',
 'direct_debit_payment_not_recognised': '28',
 'disposable_card_limits': '29',
 'edit_personal_details': '30',
 'exchange_charge':

In [18]:
results

[{'1': 'card_not_working',
  '2': 'lost_or_stolen_card',
  '3': 'card_delivery_estimate'},
 {'1': 'card_delivery_estimate',
  '2': 'card_arrival',
  '3': 'lost_or_stolen_card'},
 {'1': 'card_arrival',
  '2': 'card_delivery_estimate',
  '3': 'lost_or_stolen_card'},
 {'1': 'card_arrival',
  '2': 'card_delivery_estimate',
  '3': 'getting_physical_card'},
 {'1': 'card_arrival',
  '2': 'lost_or_stolen_card',
  '3': 'pending_card_payment'}]

In [7]:
dict1 = {0:1, 1:2, 2:3}
dict2 = {3:4, 4:5}

In [8]:
dict1 | dict2

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

In [9]:
with open("results/full_banking77_top3_0-57.json", "r") as f:
    dict57 = json.load(f)

In [15]:
len(dict57)
testdict = dict57|dict1

In [5]:
os.listdir("results/")

['.ipynb_checkpoints',
 'full_banking77_top3_319-339.json',
 'full_banking77_top3_58-117.json',
 'full_banking77_top3_118-318.json',
 'full_banking77_top3_952-3079.json',
 'banking77_top3processed.json',
 'full_banking77_top3_340-441.json',
 'full_banking77_top3_0-57.json',
 'full_banking77_top3_667-951.json',
 'full_banking77_top3_442-666.json',
 'banking77_top3.json']

In [18]:
full_results = {}
for file in os.listdir("results/"):
    if "full_banking77_top3" in file:
        with open("results/"+file, "r") as f:
            cur_results = json.load(f)
        full_results = full_results|cur_results


In [35]:
# with open("results/full_banking77_top3.json", "w") as f:
#     json.dump(full_results, f, indent=4)

In [65]:
def check_correct_format(intent2label, preds):
    for pred in preds:
        if pred not in intent2label:
            print(False)

preds = "Top_up_by_card_charge, Automatic_top_up, Top_up_limits".lower().replace(",", "").split(" ")
check_correct_format(intent2label, preds)

In [93]:
intent2label 

{'activate_my_card': '0',
 'age_limit': '1',
 'apple_pay_or_google_pay': '2',
 'atm_support': '3',
 'automatic_top_up': '4',
 'balance_not_updated_after_bank_transfer': '5',
 'balance_not_updated_after_cheque_or_cash_deposit': '6',
 'beneficiary_not_allowed': '7',
 'cancel_transfer': '8',
 'card_about_to_expire': '9',
 'card_acceptance': '10',
 'card_arrival': '11',
 'card_delivery_estimate': '12',
 'card_linking': '13',
 'card_not_working': '14',
 'card_payment_fee_charged': '15',
 'card_payment_not_recognised': '16',
 'card_payment_wrong_exchange_rate': '17',
 'card_swallowed': '18',
 'cash_withdrawal_charge': '19',
 'cash_withdrawal_not_recognised': '20',
 'change_pin': '21',
 'compromised_card': '22',
 'contactless_not_working': '23',
 'country_support': '24',
 'declined_card_payment': '25',
 'declined_cash_withdrawal': '26',
 'declined_transfer': '27',
 'direct_debit_payment_not_recognised': '28',
 'disposable_card_limits': '29',
 'edit_personal_details': '30',
 'exchange_charge':

In [201]:
def correcting_phrasing(pred, correctionfile="config/corrections.txt"):
    with open(correctionfile, "r") as f:
        lines = f.readlines()
    corrections = [correction.replace("\n", "").split(",") for correction in lines]
    for pairs in corrections:
        pred = pred.replace(pairs[0], pairs[1])
    return pred

correctionfile="config/corrections.txt"

def process_top3_intents(load_path, save_path):
    with open(load_path, "r") as f:
        results = json.load(f)
        
    full_results = results.copy()
    count = 0
    test_results = {}
    for index, result in results.items():
        topn = 1
        temp_results = {}
        pred = result["pred"].lower()
        pred = pred.replace("\n", " ")
        pred = pred.replace(",", " ")
        pred = pred.replace(".", " ")
        pred = pred.replace(":", " ")
        pred = pred.replace(")", " ")
        pred = correcting_phrasing(pred, correctionfile)
        if "_" in pred:
            pred = pred.split(" ")
            for word in pred:
                if "_" in word or word == "none":
                    temp_results["top"+str(topn)] = word
                    topn += 1
            if len(temp_results) != 3:
                count += 1
                print(f"""Predictions at index {index}: "{" ".join(pred)}" formatting isn't handled correctly\n""")

        else:
            pred = pred.split(" ")
            if pred[-1] == "":
                pred = pred[:-1]
            full_intent = ""
            for word in pred:
                if word not in ["", "1", "2", "3"]:
                    full_intent += word + "_"
                else:
                    temp_results["top"+str(topn)] = full_intent[:-1]
                    if full_intent != "":
                        topn += 1
                    full_intent = ""
            temp_results["top"+str(topn)] = full_intent[:-1]

            if len(temp_results) != 3:
                count += 1
                print(f"""Predictions at index {index}: "{" ".join(pred)}" formatting isn't handled correctly\n""")
                temp_results = {"top1":"none", "top2":"none", "top3":"none"}
        
        full_results[index]["processed_pred"] = temp_results
        
    with open(save_path, "w") as f:
        json.dump(full_results, f, indent=4)
    
    return full_results

In [200]:
load_path = "results/full_banking77_top3.json"
save_path = "results/full_banking77_top3_processed.json"

process_top3_intents(load_path, save_path)

Predictions at index 1516: "the sentence provided does not match any of the intents  as it does not mention any issue related to the banking/financial services mentioned in the intents  therefore  a list of likely intents cannot be generated" formatting isn't handled correctly

Predictions at index 1665: "the provided sentence does not match any of the given intents  please provide a valid sentence related to one of the provided intents for intent classification" formatting isn't handled correctly

Predictions at index 1898: "based on the given intents  it is unclear what the intent of the sentence is and therefore cannot be classified" formatting isn't handled correctly



{}

In [191]:
test_results["0"]

{'top1': 'card_delivery_estimate',
 'top2': 'card_about_to_expire',
 'top3': 'card_arrival'}

In [189]:
count = 0
for index, result in test_results.items():
    for topn, intent in result.items():
        if intent not in intent2label:
            count += 1
            print(index, intent, "--", full_results[index]["gold_intent"])
            
print(f"""There are {count} incorrect intent predicted""")

165 refund_not_showing_up -- extra_charge_on_statement
169 incorrect_charge_on_statement -- extra_charge_on_statement
169 refund_not_showing_up -- extra_charge_on_statement
174 dispute_charge -- extra_charge_on_statement
174 query_specific_charge -- extra_charge_on_statement
261 exchange_currency -- fiat_currency_support
293 delivery_options -- card_delivery_estimate
1017 unsupported_currency -- top_up_reverted
1037 cancelled_top_up -- top_up_reverted
1081 disputed_card_payment -- card_payment_not_recognised
1088 disputed_transaction -- card_payment_not_recognised
1094 disputed_transaction -- card_payment_not_recognised
1096 fraud_detection -- card_payment_not_recognised
1099 fraudulent_activity -- card_payment_not_recognised
1101 dispute_transaction -- card_payment_not_recognised
1103 frozen_account -- card_payment_not_recognised
1107 unauthorized_transaction -- card_payment_not_recognised
1108 disputed_transaction -- card_payment_not_recognised
1111 disputed_transaction -- card_payme

In [202]:
with open("results/full_banking77_top3_processed.json", "r") as f:
    processed = json.load(f)

In [212]:
results = {"top1":0, "top2":0, "top3":0, "others":0}
wrong_intents = {}
for index, sample in processed.items():
    gold = sample["gold_intent"]
    processed_preds = sample["processed_pred"]
    found_correct = False
    for topn, intent in processed_preds.items():
        if intent == gold:
            found_correct = True
            results[topn] += 1
    if not found_correct:
        print(index, sample["text"], gold, "--", processed_preds, "\n")
        if gold not in wrong_intents:
            wrong_intents[gold] = 1
        else:
            wrong_intents[gold] += 1
        results["others"] += 1

333 where is theft-top option? automatic_top_up -- {'top1': 'lost_or_stolen_phone', 'top2': 'lost_or_stolen_card', 'top3': 'card_delivery_estimate'} 

339 How do I make sure I don't run out of money on my card while traveling? automatic_top_up -- {'top1': 'top_up_limits', 'top2': 'cash_withdrawal_not_recognised', 'top3': 'exchange_via_app'} 

62 I found my card, I would like to reactivate it. card_linking -- {'top1': 'activate_my_card', 'top2': 'card_not_working', 'top3': 'lost_or_stolen_card'} 

64 Can I reactivate my lost card that I found this morning in my jacket pocket? card_linking -- {'top1': 'activate_my_card', 'top2': 'lost_or_stolen_card', 'top3': 'card_delivery_estimate'} 

72 Can I reactivate a card I thought I lost? card_linking -- {'top1': 'lost_or_stolen_card', 'top2': 'activate_my_card', 'top3': 'card_not_working'} 

76 Could you help me reactivate my card? It was previously lost, but I found it this morning in my jacket. card_linking -- {'top1': 'activate_my_card', 'to

In [213]:
wrong_intents

{'automatic_top_up': 5,
 'card_linking': 9,
 'card_payment_wrong_exchange_rate': 7,
 'extra_charge_on_statement': 4,
 'pending_cash_withdrawal': 8,
 'fiat_currency_support': 2,
 'card_delivery_estimate': 7,
 'top_up_reverted': 14,
 'balance_not_updated_after_cheque_or_cash_deposit': 2,
 'card_payment_not_recognised': 12,
 'why_verify_identity': 2,
 'unable_to_verify_identity': 1,
 'get_physical_card': 38,
 'visa_or_mastercard': 7,
 'topping_up_by_card': 32,
 'disposable_card_limits': 3,
 'compromised_card': 5,
 'atm_support': 1,
 'direct_debit_payment_not_recognised': 29,
 'declined_cash_withdrawal': 14,
 'pending_card_payment': 8,
 'lost_or_stolen_phone': 4,
 'request_refund': 2,
 'declined_transfer': 10,
 'Refund_not_showing_up': 1,
 'declined_card_payment': 2,
 'pending_transfer': 1,
 'terminate_account': 1,
 'card_swallowed': 2,
 'transaction_charged_twice': 1,
 'verify_source_of_funds': 3,
 'transfer_timing': 4,
 'reverted_card_payment?': 30,
 'beneficiary_not_allowed': 31,
 'tran

In [205]:
results

{'top1': 1931, 'top2': 371, 'top3': 194, 'others': 584}

In [224]:
threshold = 10

total_count = 0
for intent, count in wrong_intents.items():
    if count > threshold:
        total_count += count
        print(intent, count)
        
print(f"""Sample where more than half were wrong counted for: {total_count}""")

top_up_reverted 14
card_payment_not_recognised 12
get_physical_card 38
topping_up_by_card 32
direct_debit_payment_not_recognised 29
declined_cash_withdrawal 14
reverted_card_payment? 30
beneficiary_not_allowed 31
receiving_money 22
failed_transfer 12
transfer_into_account 17
order_physical_card 16
virtual_card_not_working 13
wrong_exchange_rate_for_cash_withdrawal 13
balance_not_updated_after_bank_transfer 14
cash_withdrawal_not_recognised 20
card_arrival 14
cancel_transfer 13
top_up_by_bank_transfer_charge 28
Sample where more than half were wrong counted for: 382


In [225]:
results_perc = {}
total = 3080
for topn, correct_count in results.items():
    results_perc[topn] = correct_count*100 / total
    
results_perc

{'top1': 62.6948051948052,
 'top2': 12.045454545454545,
 'top3': 6.298701298701299,
 'others': 18.961038961038962}

In [226]:
#Accuracy calculated without the overwhelmingly bad labels

results_perc = {}
total = 3080 - total_count
for topn, correct_count in results.items():
    if topn == "others":
        correct_count -= total_count
        results_perc[topn] = correct_count*100 / total
    else:
        results_perc[topn] = correct_count*100 / total
    
results_perc

{'top1': 71.57153446997776,
 'top2': 13.750926612305411,
 'top3': 7.190511489992587,
 'others': 7.48702742772424}